# Scraping List of Nuclear installations incident notifications — Nuclear Safety Authority (ASN) 

## *Scraping/Parsing/Cleaning/ HTML tags over multiple web pages*

The data will be extracted from HTML tags stored inside div containers over multiple pages and the [list of nuclear installations incident notifications](https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires) compiled by the Nuclear Safety Authority (Autorité de sûreté nucléaire — ASN) will be converted into a data frame that will be saved as a CSV filr.



In [1]:
# Import dependencies
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
import re
from datetime import datetime

In [2]:
# Set pandas display options (optional)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", -1)

In [3]:
# Define the URL to scrape
URL = "https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires"

# Define the headers the scraper will pass to make it look like a browser
HEADERS = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0',}

# Use `requests` to  fetch the URL
page = requests.get(URL, headers=HEADERS)

# Store the page content in a Python variable
html = page.content

In [4]:
# Use BeautifulSoup to parse the page
soup = BeautifulSoup(html, "html.parser")

# Use BeautifulSoup to find the div containers in our parsed HTML
div = soup.find_all("div", {"class": "Teaser"})
print(div)

[<div class="Teaser">
<p class="Teaser-infos Teaser-infos--grey">Publié le 09/09/2020</p>
<p class="Teaser-infos Teaser-infos--black"><span class="Teaser-infos--bold">Centrale nucléaire de Cattenom</span> - Réacteurs de 1300 MWe -
                EDF</p>
<a class="Teaser-title" href="https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Non-respect-de-la-conduite-a-tenir-prevue-par-les-regles-generales-d-exploitation-du-reacteur-3">Non-respect de la conduite à tenir prévue par les règles générales d’exploitation du réacteur 3 </a>
<p>Le 1er septembre 2020, l’exploitant de la centrale nucléaire de Cattenom a déclaré à l’ASN un événement significatif pour la sûreté relatif au non-respect de la conduite à tenir prévue par les règles générales d’exploitation (RGE) du réacteur 3 concernant le délai de réparation d’un système de contrôle commande.</p>
</div>, <div class="Teaser">
<p class="Teaser-infos Teaser-infos--grey">Publié le 07/09/2020</p>
<p

In [5]:
# Make an empty list to hold data for a CSV
list_of_rows = []

# We'll be scraping multiple pages, so start tracking what page we're on
page_num = 1

# Set up our loop to run until we tell it to stop
more_pages = True

Test the scraper over the first page and print the title of incidents listed.

In [6]:
# Loop through the rows in our current div containers using BeautifulSoup
for row in div:

    # Loop through each paragraph tag in the row
    tags = row.find_all(["p", "a"])
    #print(tags)

    date_publication = tags[0].text.strip().split(" ")[-1]
    #print(date_publication)
    
    # Loop through each paragraph row, parse and clean the tags
    for tags in row.find_all("p", {"class": "Teaser-infos Teaser-infos--black"}):
        #print(tags.text)
        centrale_nucleaire = tags.text.strip().split(" -")[0]
        #print(centrale_nucleaire)
        caracteristique = tags.text.strip().split(" -")[1]
        #print(caracteristique)
        operateur = tags.text.strip().replace(" ", "").replace("\n", "").split("-")[-1]
        #print(operateur)
    for tags in row.find_all("p", attrs={"class": None}):
        description = tags.text.strip()
        #print(description)
        date_de_rapport = re.findall("[0-9].*?[0-9]{4}", tags.text.strip())
        if len(date_de_rapport) > 0:
            date_de_rapport = date_de_rapport[0]
            #print(date_de_rapport)
    for tags in row.find_all("a"):
        titre = tags.text.strip()
        print(titre)
        hyperlien = tags["href"]
        #print(hyperlien)

Non-respect de la conduite à tenir prévue par les règles générales d’exploitation du réacteur 3
Détection tardive de l’indisponibilité simultanée de deux pompes
Non-respect des règles générales d’exploitation
Non-respect des spécifications techniques d’exploitation du réacteur 1 de Belleville
Détection tardive du non-fonctionnement d’une chaine de mesure de la radioactivité
Non-respect d’une mesure compensatoire liée à une modification temporaire des RGE
Non-respect des règles générales d’exploitation du réacteur 3 de la centrale de Chinon
Dégradation du joint d’étanchéité de la porte du bâtiment d’entreposage du combustible du réacteur 2
Indisponibilité du circuit de refroidissement intermédiaire du réacteur 3
Détection tardive de l’indisponibilité d’un groupe électrogène de secours à moteur diesel
Contamination corporelle externe ayant entraîné une exposition supérieure [...]
Indisponibilité prolongée de la turbine à combustion du site - Réacteurs 1 et 2
Contamination corporelle exte

Run the scraper over all the pages.

In [7]:
while more_pages is True:

    # Loop through the rows in our current div containers using BeautifulSoup
    for row in div:

        # Loop through each paragraph tag in the row
        tags = row.find_all(["p", "a"])
        #print(tags)
        date_publication = tags[0].text.strip().split(" ")[-1]
        #print(date_publication)
        for tags in row.find_all("p", {"class": "Teaser-infos Teaser-infos--black"}):
            #print(tags.text)
            centrale_nucleaire = tags.text.strip().split(" -")[0]
            #print(centrale_nucleaire)
            caracteristique = tags.text.strip().split(" -")[1]
            #print(caracteristique)
            operateur = tags.text.strip().replace(" ", "").replace("\n", "").split("-")[-1]
            #print(operateur)
        for tags in row.find_all("p", attrs={"class": None}):
            description = tags.text.strip()
            #print(description)
            date_de_rapport = re.findall("[0-9].*?[0-9]{4}", tags.text.strip())
            if len(date_de_rapport) > 0:
                date_de_rapport = date_de_rapport[0]
                #print(date_de_rapport)
        for tags in row.find_all("a"):
            titre = tags.text.strip()
            #print(titre)
            hyperlien = tags["href"]
            #print(hyperlien)

        list_of_tags = [date_publication, centrale_nucleaire, caracteristique, operateur, titre, description, date_de_rapport, hyperlien]
        #print(list_of_rows)
        
        list_of_rows.append(list_of_tags)
    
    if len(soup.find_all('a', {"class": "Pagination-next"}, text="Suivant >")) > 0:
        #print("There are more pages")
        
        # There is another page so fetch a new set of divs and send it back through the loop
        # Adjust the URL we're scraping by incrementing the page number
        page_num += 1
        NEXT_URL = URL + r"/(page)/" + str(page_num)
        #print(NEXT_URL)
        
        # Use requests to fetch the new URL
        page = requests.get(NEXT_URL, headers=HEADERS)
        
        # As above, get the page content into Python
        # then use BeautifulSoup to parse it and find our set of divs
        html = page.content
        soup = BeautifulSoup(html, "html.parser")
        div = soup.find_all("div", {"class": "Teaser"})
        
        # Pause for a second to avoid overwhelming the server
        time.sleep(1)
        
    else:
        more_pages = False

In [8]:
list_of_rows

[['09/09/2020',
  'Centrale nucléaire de Cattenom',
  ' Réacteurs de 1300 MWe',
  'EDF',
  'Non-respect de la conduite à tenir prévue par les règles générales d’exploitation du réacteur 3',
  'Le 1er septembre 2020, l’exploitant de la centrale nucléaire de Cattenom a déclaré à l’ASN un événement significatif pour la sûreté relatif au non-respect de la conduite à tenir prévue par les règles générales d’exploitation (RGE) du réacteur 3 concernant le délai de réparation d’un système de contrôle commande.',
  '1er septembre 2020',
  'https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Non-respect-de-la-conduite-a-tenir-prevue-par-les-regles-generales-d-exploitation-du-reacteur-3'],
 ['07/09/2020',
  'Centrale nucléaire de Chooz B',
  ' Réacteurs de 1450 MWe',
  'EDF',
  'Détection tardive de l’indisponibilité simultanée de deux pompes',
  'Le 2 septembre 2020, l’exploitant de la centrale nucléaire de Chooz a déclaré à l’ASN un événement signific

In [9]:
# Use Python's CSV library to create the output file
outfile = open("asn-incidents-installations-nucleaires.csv", "w", newline="", encoding="utf-8")
writer = csv.writer(outfile, delimiter=",")
writer.writerow(["date_publication", "centrale_nucleaire", "caracteristique", "operateur", "titre", "description", 
                 "date_de_rapport", "hyperlien"])
writer.writerows(list_of_rows)
outfile.close()

In [10]:
df = pd.read_csv("asn-incidents-installations-nucleaires.csv")

df.head()

,date_publication,centrale_nucleaire,caracteristique,operateur,titre,description,date_de_rapport,hyperlien
0,09/09/2020,Centrale nucléaire de Cattenom,Réacteurs de 1300 MWe,EDF,Non-respect de la conduite à tenir prévue par les règles générales d’exploitation du réacteur 3,"Le 1er septembre 2020, l’exploitant de la centrale nucléaire de Cattenom a déclaré à l’ASN un événement significatif pour la sûreté relatif au non-respect de la conduite à tenir prévue par les règles générales d’exploitation (RGE) du réacteur 3 concernant le délai de réparation d’un système de contrôle commande.",1er septembre 2020,https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Non-respect-de-la-conduite-a-tenir-prevue-par-les-regles-generales-d-exploitation-du-reacteur-3
1,07/09/2020,Centrale nucléaire de Chooz B,Réacteurs de 1450 MWe,EDF,Détection tardive de l’indisponibilité simultanée de deux pompes,"Le 2 septembre 2020, l’exploitant de la centrale nucléaire de Chooz a déclaré à l’ASN un événement significatif pour la sûreté relatif à la détection tardive de l’indisponibilité simultanée de deux pompes du système d’alimentation de secours des générateurs de vapeur (ASG) du réacteur 1.",2 septembre 2020,https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Detection-tardive-de-l-indisponibilite-simultanee-de-deux-pompes
2,04/09/2020,Centrale nucléaire de Gravelines,Réacteurs de 900 MWe,EDF,Non-respect des règles générales d’exploitation,"Le 28 août 2020, l’exploitant de la centrale nucléaire de Gravelines a déclaré à l’Autorité de sûreté nucléaire un événement significatif pour la sûreté relatif au non-respect des règles générales d’exploitation concernant l’indisponibilité d’un circuit de ventilation de l’enceinte de confinement du réacteur 1.",28 août 2020,https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Non-respect-des-regles-generales-d-exploitation32
3,20/08/2020,Centrale nucléaire de Belleville-sur-Loire,Réacteurs de 1300 MWe,EDF,Non-respect des spécifications techniques d’exploitation du réacteur 1 de Belleville,"Le 18 août 2020, l’exploitant de la centrale nucléaire de Belleville-sur-Loire a déclaré à l’ASN un événement significatif pour la sûreté, relatif à l’indisponibilité, pendant plus d’une heure et demi, de certains systèmes contribuant à la sûreté du réacteur.",18 août 2020,https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Non-respect-des-specifications-techniques-d-exploitation-du-reacteur-1-de-Belleville
4,19/08/2020,Centrale nucléaire du Blayais,Réacteurs de 900 MWe,EDF,Détection tardive du non-fonctionnement d’une chaine de mesure de la radioactivité,"Le 11 août 2020, l’exploitant de la centrale nucléaire du Blayais a déclaré à l’Autorité de sûreté nucléaire (ASN) un événement significatif pour la sûreté relatif à l’indisponibilité d’une chaîne de mesure de la radioactivité de l’enceinte de confinement, du réacteur 4 alors qu’il procédait à la décompression de cette enceinte.",11 août 2020,https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Detection-tardive-du-non-fonctionnement-d-une-chaine-de-mesure-de-la-radioactivite


In [11]:
df.tail()

,date_publication,centrale_nucleaire,caracteristique,operateur,titre,description,date_de_rapport,hyperlien
2311,11/01/2000,Centrale nucléaire de Fessenheim,Réacteurs de 900 MWe,EDF,Inspection de revue,"La révision décennale de la centrale nucléaire de Fessenheim a été l'occasion d'une opération de contrôle d'un type nouveau : "" l'inspection de revue "".",[],https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Inspection-de-revue
2312,06/01/1981,Centrale nucléaire de Fessenheim,Réacteurs de 900 MWe,EDF,Évènements significatifs classés au niveau 3 sur l'échelle INES,"2008 : irradiation par une source de cobalt 60 d’un travailleur intervenant dans un bunker d’irradiation sur le site de l’ONERA de Toulouse.\r\n(Incident classé hors installations nucléaires) \r\n\r\n2002 : incident classé par l’Autorité compétente suédoise, lors du transport par la société Federal Express, entre la Suède et les États-Unis via l’aéroport de Roissy, d’un colis présentant à son arrivée un débit de dose supérieur à la limite réglementaire admissible.\r\n(Incident classé hors installations nucléaires)\r\n\r\n1981 : incendie d’un silo de stockage à La Hague.",[],https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Evenements-significatifs-classes-au-niveau-3-sur-l-echelle-INES
2313,13/03/1980,Centrale nucléaire de Fessenheim,Réacteurs de 900 MWe,EDF,Événements significatifs classés au niveau 4 sur l'échelle INES,"En France, en 1969 et 1980 : endommagement du cœur du réacteur A2 de Saint-Laurent.\r\nA l'étranger, en 1999 : accident de criticité dans une installation de fabrication de combustible de Tokai-Mura au Japon, avec irradiation aiguë de trois travailleurs dont deux sont décédés. En 1973 : rejet de matières radioactives à la suite d'une réaction exothermique dans un réservoir du procédé de retraitement à l'usine Windscale (Royame-Uni).",1969 et 1980,https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Evenements-significatifs-classes-au-niveau-4-sur-l-echelle-INES
2314,28/03/1979,Centrale nucléaire de Fessenheim,Réacteurs de 900 MWe,EDF,Évènements significatifs classés au niveau 5 sur l'échelle INES,En France : aucun. A l'étranger : 1979 : Fusion partielle du cœur du réacteur à Three Mile Island aux États-Unis.,[],https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Evenements-significatifs-classes-au-niveau-5-sur-l-echelle-INES
2315,29/09/1957,Centrale nucléaire de Fessenheim,Réacteurs de 900 MWe,EDF,Évènements significatifs classés au niveau 6 sur l'échelle INES,En France : aucun. A l'étranger : 1957 : Explosion d'une cuve de produits radioactifs à l'usine de retraitement de Kyshtym en URSS.,[],https://www.asn.fr/Controler/Actualites-du-controle/Avis-d-incident-des-installations-nucleaires/Evenements-significatifs-classes-au-niveau-6-sur-l-echelle-INES
